# TEAM PARIS P6 - Sepsis Illness Classification Project

## Introduction/Background


## Business Understanding

## Data Understanding

In [ ]:
#### Hypothesis & Questions 
#### Null Hypothesis: H0: There is no relationship between high Body Mass Index and sepsis.
#### Alternate Hypothesis: H1: There is a relationship between high Body Mass Index and sepsis.
####



#### Assumptions:
1. It was assumed that the blood pressure used was the diastolic type.


#### Questions

1. How many patients are under weight, have healthy weight, over weight, obesed and severe obesed?
2. What is the distribution of ages for patients captured in the data?
3. How many of the patients fall under the Normal, Elevated and High Blood Pressure?
4. Is Body Mass Index affected by Age?
5. Is Blood Pressure affected by Age?
6. What is the relationship between Age and Body In
7. How many patients have the tendency of developing sepsis?
8. Which age group are more prone to developing sepsis?
9. Does having an insurance enhance patients chances of getting sepsis?
10. Is body mass directly correlated with a patients tendency to get sepsis?
11. Does the blood parameters associated with sepsis?